
Este teste visa testar suas habilidades estatísticas e de programação. Junto com este notebook, foi disponibilizado uma base `.csv` que está pré-carregada através do código abaixo em Python 3. 

Para resolver o teste, crie novos blocos de código e/ou markdown após a pergunta. Se preferir, pode usar R ou outra linguagem de sua preferência para fazer o teste, contanto que consiga salvar e rodar no Jupyter Notebook - para mais informações sobre como instalar outras linguagens no Jupyter, [clique aqui](http://jupyter.readthedocs.io/en/latest/install-kernel.html). Ao terminar o teste, responda o email com o arquivo do notebook editado e salvo com o seu nome. 

## import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from sklearn import externals
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import pickle

import lime
import lime.lime_tabular

from yellowbrick.regressor import ResidualsPlot
from fancyimpute import SoftImpute

import warnings
warnings.filterwarnings('ignore')

## functions & var

In [ ]:
modelos = [
    GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor,
    SGDRegressor, Lasso, Ridge, ElasticNet, LinearRegression,
]

def fnc_Dummies(df):
    for cat_feature in df.select_dtypes(include=['object']).columns:
        df[cat_feature] = pd.Categorical(df[cat_feature]).codes
        df[cat_feature] = df[cat_feature].replace(-1,np.nan)
    return pd.DataFrame(df)

def fnc_Imputer(df):
    imp_cols = df.columns.values
    imputer = SoftImpute()
    return pd.DataFrame(imputer.fit_transform(df), columns= imp_cols)

## datasets

In [ ]:
# importando os indicadores
indicators = pd.read_csv('Indicators.csv')

# dataset de países da união européia
eu_countries = ['European Union', 'Austria','Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
                'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
                'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
                'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain',
                'Sweden', 'United Kingdom']

# definindo apenas a base de indicadores da união européia
eu = indicators[indicators['CountryName'].isin(eu_countries)]

## Questão 1 ###
Mostre graficamente a evolução da taxa de fertilidade e do Produto Interno Bruto na União Européia.

#### Resposta: 

In [ ]:
# GDP (current US$)
# https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
pib_cod = 'NY.GDP.MKTP.CD'
pib_nome = 'GDP (current US$)'

# Fertility rate, total (births per woman)
# https://data.worldbank.org/indicator/SP.DYN.TFRT.IN
fertilidade_cod = 'SP.DYN.TFRT.IN'
fertilidade_nome = 'Fertility rate, total (births per woman)'

# Filtro para UE
df_grafico = eu[eu['CountryName']=='European Union']

# Dados para gráfico
data1 = df_grafico[df_grafico['IndicatorCode']==pib_cod]['Value'].values
data1_year = df_grafico[df_grafico['IndicatorCode']==pib_cod]['Year'].values

data2 = df_grafico[df_grafico['IndicatorCode']==fertilidade_cod]['Value'].values
data2_year = df_grafico[df_grafico['IndicatorCode']==fertilidade_cod]['Year'].values

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Year')
ax1.set_ylabel(pib_nome, color=color)
ax1.plot(data1_year, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel(fertilidade_nome, color=color)  
ax2.plot(data2_year, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.show()

## Questão 2 ###
Podemos afirmar que existe uma correlação entre a taxa de fertilidade e do Produto Interno Bruto na União Européia? Explique.

#### Resposta: 

### Sim, existe uma correlação, mas é negativa. A taxa de fertilidade da UE é inversamente proporcional ao PIB, ou seja, quanto maior é o PIB menor é a taxa.

In [ ]:
# Anos com valores
# df_grafico[df_grafico['IndicatorCode'].isin([fertilidade_cod,pib_cod])].groupby('Year').count()

In [ ]:
df_lr = df_grafico[df_grafico['Year']<2014]

x = df_lr[df_lr['IndicatorCode']==fertilidade_cod]['Value'].values
y = df_lr[df_lr['IndicatorCode']==pib_cod]['Value'].values

fit = np.polyfit(x,y,1)
fit_fn = np.poly1d(fit)

plt.title(fertilidade_nome[:14]+'... vs '+pib_nome)
plt.xlabel(pib_nome)
plt.ylabel(fertilidade_nome)

plt.plot(x,y, 'yo', x, fit_fn(x), '--k')

print('Correlação de Pearson: ',np.corrcoef(x,y)[1][0])

## Questão 3 ###
Podemos afirmar com as informações das questões anteriores que _"o crescimento do PIB levou a uma redução da taxa de fertilidade na União Européia"_?. Explique.

#### Resposta: 

### Correlação não implica causa! 

### Pouco provável, não podemos concluir que a resposta para a redução é só o PIB. Precisamos utilizar mais variáveis para tentar responder esse evento de redução, pois ele é complexo e sabemos que existem muitas variáveis envolvidas. Seria interessante utilizar modelos de machine learning ou deep learning para tentar responder esse evento com mais precisão.

## Questão 4 ###

Consumo, segundo [Keynes](https://en.wikipedia.org/wiki/John_Maynard_Keynes) (economista inglês), é uma função linear de um componente 

autônomo invariável que é independente da renda disponível (_a_) e da proporção da renda alocada para consumo (_b_) multiplicada pela renda (_Y_), conforme abaixo:

$$C = a + b * Y$$

O parâmetro _b_ é conhecido como a propensão marginal a consumir, ou seja, o crescimento no consumo decorrente de um aumento na renda disponível.

Faça a regressão da equação acima por `Mínimos Quadrados Ordinários` utilizando _Household final consumption expenditure (constant 2005 US\$)_ como proxy para o consumo, o _GDP at market prices (constant 2005 US$)_ como proxy para a renda disponível utilizando os dados de cada um dos países membros da União Européia. Podemos rejeitar a hipótese de que o parâmetro _b_ é igual a zero? Comente.



#### Resposta: 

### Sim, podemos. O consumo de fato subiu durante o crescimento da renda.
### O modelo de regressão linear apresentou R² de 0.99, assim indicando forte relação entre as variáveis utilizadas. A distribuição de erros é uma curva normal e as variáveis apresentam relação linear, esses são pré-requisitos fundamentais para um modelo linear. Seguem as evidências:

#### Preparando Dataset

In [ ]:
# Conversão do dataset
var_x = 'Household final consumption expenditure (constant 2005 US$)'
var_y = 'GDP at market prices (constant 2005 US$)'
df_mqo = eu[(~eu['CountryName'].isin(['European Union'])) & eu['IndicatorName'].isin([var_x,var_y])]

anos = df_mqo['Year'].unique()
paises = df_mqo['CountryName'].unique()
lista = [(x, y) for x in anos for y in paises]

df_mqo_v2 = pd.DataFrame(columns=['Year','CountryName',var_x,var_y],dtype=np.float)

df_mqo_v2['Year'] = np.asarray([ano for ano,pais in lista])
df_mqo_v2['CountryName'] = np.asarray([pais for ano,pais in lista])

for index, row in df_mqo.iterrows():
    if row['IndicatorName'] == var_x:
        df_mqo_v2.loc[(df_mqo_v2['Year'] == row['Year']) & (df_mqo_v2['CountryName'] == row['CountryName']), var_x] = row['Value']
    else:
        df_mqo_v2.loc[(df_mqo_v2['Year'] == row['Year']) & (df_mqo_v2['CountryName'] == row['CountryName']), var_y] = row['Value']

#### Modelo Linear - Resultados

In [ ]:
# Faz cópia de dataset
df_mqo_v3 = df_mqo_v2.dropna().copy()

# Converte variáveis categóricas
df_mqo_v3 = fnc_Dummies(df=df_mqo_v3)

# Variáveis para treino
feature_names = ['Year', 'CountryName', var_x]
target_name = var_y
X = df_mqo_v3[feature_names]
y = df_mqo_v3[target_name]

# Separa dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Prepara modelo para gráfico
model = LinearRegression()
visualizer = ResidualsPlot(model)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.poof()

#### Correlação

In [ ]:
x = df_mqo_v3[var_x].values
y = df_mqo_v3[var_y].values

fit = np.polyfit(x,y,1)
fit_fn = np.poly1d(fit)

plt.title('Regressão linear')
plt.xlabel(var_x)
plt.ylabel(var_y)

plt.plot(x,y, 'yo', x, fit_fn(x), '--k')

print('Correlação de Pearson: ',np.corrcoef(x,y)[1][0])

## Questão 5

Utilizando todas variáveis disponíveis na base de dados, construa o melhor modelo para explicar a taxa de fertilidade com até 5 variáveis. 


#### Resposta: 

### Data preparation & Missing Values

In [ ]:
# Cópia de dataframe
df_new = eu[(~eu['CountryName'].isin(['European Union']))]
df_new.Year.astype('category')

# Pivot table para Indicadores
df_new_v2 = pd.pivot_table(df_new, values='Value', index=['Year','CountryName'], columns='IndicatorName')
df_new_v2.insert(loc=0, column='CountryName', value=df_new_v2.index.get_level_values('CountryName') )
df_new_v2.insert(loc=0, column='Year', value=df_new_v2.index.get_level_values('Year') )

# Converte variáveis categóricas
df_new_v2 = fnc_Dummies(df=df_new_v2)

In [ ]:
# Elimina features com menos de 30% de dados
df = df_new_v2.copy()
df_new_v3 = df[[column for column in df if df[column].count() / len(df) >= 0.3]]

print("Lista de fatos excluídos:\n\n Total excluído:", round(len(df_new_v3.columns)/len(df.columns),2)*100,'%\n\n', end=" ")

for c in df.columns:
    if c not in df_new_v3.columns:
        print(c, end="\n ")

# Missing Values
df_new_v3 = fnc_Imputer(df_new_v2)

In [ ]:
# Matriz de Correlação da Fertilidade
df_corr = df_new_v3.corr()
df_corr[[fertilidade_nome]].sort_values(by=fertilidade_nome,ascending=False)

In [ ]:
# Variável de interesse
var_interesse = fertilidade_nome

# Escopo de treino/test
colunas = list(df_new_v3.columns)
colunas.remove(fertilidade_nome)
feature_names = colunas
target_name = var_interesse
X = df_new_v3[feature_names]
y = df_new_v3[target_name]

# Separa dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Lista de resultados
resultados = [['status','var','model','mean','std','time','scores','caminho']]

# Score dos modelos
print('='*100)
print(var_interesse)
print('='*100)

Y = df_new_v3[[var_interesse]].values.ravel()

for var in modelos:
    start = time.time()
    try:
        filename = str('../../models/'+var.__name__+'.model')
        print(var)
        clf = var()

        # Calcula o score
        scores = cross_val_score(clf, df_new_v4, Y, cv=10, error_score='raise')
        print('Mean score: ',np.mean(scores), '/ Std Score: ',np.std(scores))
        resultados.append(['ok',var_interesse,var.__name__,np.mean(scores),np.std(scores),time.time() - start,scores,filename])

        # Salva modelo            
        model = clf.fit(X_train, y_train)
        externals.joblib.dump(model,filename)

    except(Exception):
        print('>> Validar parâmetros.')
        resultados.append(['erro',var_interesse,var.__name__,None,None,time.time() - start,None,None])
        pass
    finally:            
        print('-'*100)

In [ ]:
# Salva planilhas com score de modelos
writer = pd.ExcelWriter('../../dist/resultados_modelos.xlsx', engine='xlsxwriter')
df_final = pd.DataFrame(resultados[1:])
df_final.columns = resultados[0]
df_final.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()

In [ ]:
df_final.sort_values(by=['mean'], ascending=False)

In [ ]:
# Carrega melhor modelo
model = externals.joblib.load(df_final.sort_values(by=['mean'], ascending=False)['caminho'][0])

# Acurácia do modelo de regressão
model.score(X_test, y_test)

In [ ]:
# Preparando modelo Lime
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train.values,
    feature_names=list(X_train.columns), 
    class_names=[var_interesse],
    verbose=True, 
    mode='regression'
)

In [ ]:
# Define seed para modelo
def explain(instance, predict_fn, **kwargs):
  np.random.seed(16)
  return explainer.explain_instance(instance, predict_fn, **kwargs)

# Modelo de predição para teste
i = 302
exp = explain(X_test.values[i], model.predict, num_features=5)

In [ ]:
# Resultados para 5 features
exp.show_in_notebook(show_table=True)

In [ ]:
# Detalhe por variável
exp.as_list()